In [11]:
# 导入函数库和数据
import numpy as np
from scipy.integrate import odeint # 微分方程函数
from scipy.optimize import minimize # 优化函数
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import math
sns.set_style("darkgrid") # 设置图表风格
sns.set_context("paper")
plt.rcParams["font.sans-serif"]=["SimHei"] # 解决中文标题，坐标轴名，刻度名等问题
plt.rcParams["axes.unicode_minus"]=False  
import openpyxl # 用于打开excel表格
import palettable # 用于打开excel子表

df1 = pd.read_excel("data/长春交通与小区网络.xlsx","交通路口节点数据")
df2 = pd.read_excel("data/长春交通与小区网络.xlsx","各区主要小区数据")

In [13]:
# 预备工作：预估每个路口所属区域，方法为看每个路口离哪一个小区最近，就把它判给该小区所属区
District = []
length1 = len(df1)
length2 = len(df2)
for i in range(length1):
    mini_dis = 1000000000  # 最短距离
    mini_qu = None # 最近的小区
    for j in range(length2):
        distance = (df1.iloc[i,1]-df2.iloc[j,4])*(df1.iloc[i,1]-df2.iloc[j,4]) + (df1.iloc[i,2]-df2.iloc[j,5])*(df1.iloc[i,2]-df2.iloc[j,5])
        if distance<=mini_dis:
            mini_qu = df2.iloc[j,7]
            mini_dis = distance
    District.append(mini_qu)

District = pd.DataFrame(District)
District.to_csv("data/交通路口预估所属区域.csv",header=False)

In [12]:
# 问题2
# 计算每个区总人口数（已给定），总小区数，总路口数，以及4月4日-4月13日新增感染人数
Crossings = {} # 路口数字典
Communities = {} # 小区数字典
length1 = len(df1)
length2 = len(df2)

"""
人口数汇总（单位：万人）
朝阳区：57.8
南关区：48.9
宽城区：38.5
绿园区：42.6
二道区：32.6
长春新区(高新)：36.8
经开区：20.3
净月区：22.8
汽开区：21.7

注意：附表2中所给宽城区、绿园区、二道区隔离人数有误，已依据附件5修正！
"""
# 路口数汇总
for i in range(length1):
    d = df1.iloc[i,3] # 区域名
    if d in Crossings:
        Crossings[d] = Crossings[d]+1
    else:
        Crossings[d] = 1
print(Crossings)
"""
{'经开区': 590, '绿园区': 1080, '朝阳区': 1095, '南关区': 1341, '长春新区(高新)': 970, '汽开区': 1324, '宽城区': 1306, '净月区': 1042, '二道区': 1184}
"""

# 小区数汇总
for j in range(length2):
    d = df2.iloc[j,7]
    if d in Communities:
        Communities[d] = Communities[d]+1
    else:
        Communities[d] = 1
print(Communities)
"""
{'宽城区': 168, '二道区': 169, '朝阳区': 200, '绿园区': 184, '南关区': 202, '经开区': 114, '长春新区(高新)': 85, '净月区': 146, '汽开区': 141}
"""

# 每万人新增感染人数汇总
df3 = pd.read_excel("data/3月4日至4月13日长春市各区疫情动态.xlsx","新增本土感染者")
df4 = pd.read_excel("data/3月4日至4月13日长春市各区疫情动态.xlsx","新增无症状感染者")
Populations = [57.8, 48.9, 38.5, 42.6, 32.6, 36.8, 20.3, 22.8, 21.7] # 九个区隔离人口数

for i in range(1,10):
    count = 0
    for j in range(31,41):
        count = count +df3.iloc[j,i]+df4.iloc[j,i]
    print(count, count/Populations[i-1])

"""
# 每个区每万人平均感染人数（某种程度上体现蔬菜包投递后疫情防控的效果）
朝阳区:19.62
南关区:25.95
宽城区:118.13
绿园区:49.34
二道区:25.34
长春新区(高新):18.15
经开区:68.82
净月区:20.53
汽开区:12.30
"""

{'经开区': 590, '绿园区': 1080, '朝阳区': 1095, '南关区': 1341, '长春新区(高新)': 970, '汽开区': 1324, '宽城区': 1306, '净月区': 1042, '二道区': 1184}
{'宽城区': 168, '二道区': 169, '朝阳区': 200, '绿园区': 184, '南关区': 202, '经开区': 114, '长春新区(高新)': 85, '净月区': 146, '汽开区': 141}
1134.0 19.61937716262976
1269.0 25.950920245398773
4548.0 118.12987012987013
2102.0 49.34272300469483
826.0 25.337423312883434
668.0 18.15217391304348
1397.0 68.81773399014779
468.0 20.526315789473685
267.0 12.304147465437788


'\n# 每个区每万人平均感染人数（某种程度上体现蔬菜包投递后疫情防控的效果）\n朝阳区:19.62\n南关区:25.95\n宽城区:118.13\n绿园区:49.34\n二道区:25.34\n长春新区(高新):18.15\n经开区:68.82\n净月区:20.53\n汽开区:12.30\n'

In [14]:
# 问题2
# 综合评价模型：指标分别为各区【1】每万人平均投递点数X1（正） 【2】每路口平均投递点数X2（反） 【3】每小区平均投递点数X3（正）

# 指标数计算
df5 = pd.read_excel("data/综合评价表.xlsx")

df_index = pd.DataFrame(
    {
    "X1":(df5["投放点数"]/df5["隔离人口数"]).values,
    "X2":(df5["投放点数"]/df5["交通路口数"]).values,
    "X3":(df5["投放点数"]/df5["小区数"]).values,
    }
)

# 指标权重计算（熵权法）
# 1指标归一化与同向性处理
df_index["X1"] = df_index[["X1"]].apply(lambda x: x/math.sqrt((x*x).sum())) # 正向
df_index["X2"] = df_index[["X2"]].apply(lambda x: 1-(x/math.sqrt((x*x).sum()))) # 反向
df_index["X3"] = df_index[["X3"]].apply(lambda x: x/math.sqrt((x*x).sum())) # 正向

# 2计算熵权
D = np.array([]) # 信息冗余度
for i in range(1,4):
    p = np.array(df_index[["X"+str(i)]].apply(lambda x: x/sum(x)).values) # p值
    lnp = np.array([]) 
    for i in range(len(p)):
        lnp = np.append(lnp, math.log(p[i]))
    e = -(1/math.log(9))*(np.dot(p.T,lnp)) # 熵值
    D = np.append(D,1-e)

W = D/sum(D)
print(W)

# 综合得分
S = []
for j in range(9):
    score = 0
    for i in range(3):
        score = score+W[i]*df_index.iloc[j,i]
    S.append(score)

print(S/sum(S))
"""
指标分别为各区【1】每万人平均投递点数X1（正） 【2】每路口平均投递点数X2（反） 【3】每小区平均投递点数X3（正）
各指标权重：
X1：0.4723796  X2：0.05643422  X3：0.47118618

熵权法评价得分
朝阳区：0.05455709
南关区：0.12111847
宽城区：0.10798066
绿园区：0.2206423
二道区：0.02682831
长春新区(高新)：0.17666526
经开区：0.05124226
净月区：0.21159648 
汽开区：0.02936918
"""


[0.46952549 0.05869379 0.47178072]
[0.05473089 0.12032698 0.10742062 0.21828914 0.03302285 0.17534268
 0.05146127 0.20946308 0.02994249]


'\n指标分别为各区【1】每万人平均投递点数X1（正） 【2】每路口平均投递点数X2（反） 【3】每小区平均投递点数X3（正）\n各指标权重：\nX1：0.4723796  X2：0.05643422  X3：0.47118618\n\n熵权法评价得分\n朝阳区：0.05455709\n南关区：0.12111847\n宽城区：0.10798066\n绿园区：0.2206423\n二道区：0.02682831\n长春新区(高新)：0.17666526\n经开区：0.05124226\n净月区：0.21159648 \n汽开区：0.02936918\n'

In [17]:
# 问题2
# 综合评价模型：指标分别为各区【1】每万人平均投递点数X1（正） 【2】每路口平均投递点数X2（反） 【3】每小区平均投递点数X3（正）

# 指标数计算
df5 = pd.read_excel("data/综合评价表.xlsx")

def index(df5):
    df_index = pd.DataFrame(
        {
        "X1":(df5["投放点数"]/df5["隔离人口数"]).values,
        "X2":(df5["投放点数"]/df5["交通路口数"]).values,
        "X3":(df5["投放点数"]/df5["小区数"]).values,
        }
    )

    # 指标权重计算（层次分析法）
    # 1指标归一化与同向性处理
    df_index["X1"] = df_index[["X1"]].apply(lambda x: (x-x.min())/(x.max()-x.min())) # 正向
    df_index["X2"] = df_index[["X2"]].apply(lambda x: (x.max()-x)/(x.max()-x.min())) # 反向
    df_index["X3"] = df_index[["X3"]].apply(lambda x: (x-x.min())/(x.max()-x.min())) # 正向
    return df_index
df_index = index(df5)

# 2计算最大特征值和对应的特征向量
a = np.array([[1, 2/3, 3/2,], # a为自己构造的输入判别矩阵
             [3/2, 1, 3],
             [2/3, 1/3, 1]])   
w = np.linalg.eig(a)   # 返回特征值和特征向量
tzz = np.max(w[0])   # 最大特征值
t = np.argwhere(w[0]==tzz) # 寻找最大特征值所在的行和列
tzx = w[1][::-1,t[0]]    # 最大特征值对应的特征向量

# 一致性检验
RILIST = [0,0,0.58,0.9,1.12,1.24,1.32,1.41,1.45,1.49,1.52,1.54,1.56,1.58,1.59]
n = a.shape[0]
RI = RILIST[n]
CI = (tzz-n)/(n-1)
CR = CI/RI
print(CR.real) # 如果CR<=0.1, 

T = tzx/sum(tzx) # 各指标权重，取实数部分
W = [T[0].real[0], T[1].real[0], T[2].real[0]]
print(W)

# 综合得分
S = np.array([])
for j in range(9):
    score = 0
    for i in range(3):
        score = score+W[i]*df_index.iloc[j,i]
    S = np.append(S, score)

print(S/sum(S))


"""
指标分别为各区【1】每万人平均投递点数X1（正） 【2】每路口平均投递点数X2（反） 【3】每小区平均投递点数X3（正）
层次分析法各指标权重：
X1：0.1859896069849466  X2：0.5069483173576544 X3：0.3070620756573989

层次分析法评价得分
朝阳区：0.12137939
南关区：0.10756521
宽城区：0.10385762
绿园区：0.10888491
二道区：0.10567299
长春新区(高新)：0.11919366
经开区：0.10401058
净月区：0.12569955
汽开区：0.1067361
"""
"""
[0.1859896069849466, 0.5069483173576544, 0.3070620756573989]
[0.12137939 0.10756521 0.10385762 0.10888491 0.10567299 0.11919366
 0.10401058 0.12569955 0.1067361 ]
"""


0.005112618174599367
[0.1859896069849466, 0.5069483173576544, 0.3070620756573989]
[0.10137939 0.10756521 0.11385762 0.09888491 0.10567299 0.13419366
 0.10401058 0.12769955 0.1067361 ]


'\n指标分别为各区【1】每万人平均投递点数X1（正） 【2】每路口平均投递点数X2（反） 【3】每小区平均投递点数X3（正）\n层次分析法各指标权重：\nX1：0.459958  X2：0.318917 X3：0.221124\n\n层次分析法评价得分\n朝阳区：0.08125937\n南关区：0.11249421\n宽城区：0.11211006\n绿园区：0.14767371\n二道区：0.07418333\n长春新区(高新)：0.13777322\n经开区：0.08403815\n净月区：0.17425997\n汽开区：0.07620798\n'